#### Import libraries

In [1]:
import os
import sys
import pandas as pd
import yfinance as yf
import plotly as px

#### Current working directory

In [2]:
#current working directory
current_dir=os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir=os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

c:\10acadamy\week1\financial-data-analysis\notebooks
c:\10acadamy\week1\financial-data-analysis


In [6]:
from scripts.financial_analyzer import FinancialAnalyzer,GeneralConfiguration

In [8]:
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

dataframe=GeneralConfiguration.read_csv_file('../dataset/raw_analyst_ratings.csv')
dataframe.head()
# msft=yf.ticker("MSFT")
# hist=msft.history(period='max')
# fig=make_subplots(rows=1,cols=4,subplot_titles=['Close','Open','High','Low'])
# fig.add_trace(go.Scatter(x=hist.index,y=hist['Close']),row=1,col=1)
# fig.add_trace(go.Scatter(x=hist.index,y=hist['Open']),row=1,col=2)
# fig.add_trace(go.Scatter(x=hist.index,y=hist['High']),row=1,col=3)
# fig.add_trace(go.Scatter(x=hist.index,y=hist['Low']),row=1,col=4)

# fig.update_layout(height=400,width=1200,title_text="Microsoft Stock Analysis")
# fig.show()


,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A
